In [ ]:
from reinforcement_training import *
from notebook_game_helper import *

In [ ]:
# lets train a DQN model!
# make the model
hiddens = [{"size":10,"activation":"relu"},
            {"size":20,"activation":"relu"},
            {"size":10,"activation":"relu"}]
# make an optimizer
from keras.optimizers import sgd, RMSprop, Adagrad, Adadelta, Adam
# note to self: DON'T CHANGE THIS UNTIL YOU KNOW WE'RE LEARNING SOMETHING
# optimizer = sgd(lr = 0.0001)
# optimizer_str = "SGD"
# optimizer = Adagrad()
# optimizer_str = "Adagrad"
# optimizer = RMSprop()
# optimizer_str = "RMSprop"
# optimizer = Adadelta()
# optimizer_str = "Adadelta"
optimizer = Adam()
optimizer_str = "Adam"
model = baseline_model(optimizer, hiddens)

In [ ]:
# prepare the game for training model
training_game_size = 6
training_game = NeuralNaviGame(training_game_size,
                                training_game_size,
                                model,
                                model_type = "reinforcement")
training_game.setup()

In [ ]:
training_episodes = 500
steps = 20
# train the model
output = train_model(game = training_game,
                model = model,
                episodes = training_episodes,
                steps = steps)

In [ ]:
# plot learning info
title_str = str(training_game_size) + "x" + str(training_game_size) + " with " 
title_str += str(training_episodes) + " episodes, " + str(steps) + " steps per episode, & "
title_str += str(len(hiddens)) + " hidden layers, optimized with " + optimizer_str + "\n"
log = [output['log'], output['replays']]
f, axarr = pl.subplots(2, 1, figsize = (20, 10))
# f.canvas.set_window_title("RL Loss, 100 eps w/ 50 steps, Look: 20")
x = np.linspace(0, len(log[0]), len(log[0]))
axarr[0].plot(x, log[0])
axarr[0].set_title(title_str + 'Total Loss')
axarr[1].plot(x, log[1])
axarr[1].set_title('Replay Loss')
# axarr[2].plot(x, np.log(log[0]))
# axarr[2].set_title('Log Total Loss')
# axarr[3].plot(x, np.log(log[1]))
# axarr[3].set_title('Log Replay Loss')
f.subplots_adjust(hspace=0.2)

file_str = str(training_game_size) + "x" + str(training_game_size) + "_" 
file_str += str(training_episodes) + "_" + str(steps) + "_" + str(len(hiddens))
file_str += "_" + optimizer_str

pl.plot()
pl.savefig("rl_curve_" + file_str + ".png")
pl.show()

In [ ]:
ipt = training_game.Navigator.strategy.get_input(0)
predict = training_game.Navigator.strategy.get_quality()
reward = training_game.Navigator.strategy.get_reward()
print(ipt, predict, reward)

In [ ]:
# prepare the game for validating model
val_game_size = training_game_size
val_game = NeuralNaviGame(val_game_size,
                                val_game_size,
                                model,
                                model_type = "reinforcement")
val_game.setup()

In [ ]:
animate_game(val_game, n = 20)